In [1]:
from BetaShF import ShF
from BetaShM import ShM 
import numpy as np 
import matplotlib.pyplot as plt
import time

In [2]:
def logSample(prefix, sample, fitness, force, moment):
    def appendToFile(name, text): 
        with open("./experiments/"+name, "a") as f:
            f.write(text + '\n')
    s = ""
    for x in sample: s += str(x) + " "
    appendToFile(str(prefix) + "_Population.txt", s)
    appendToFile(str(prefix) + "_Fitness.txt", str(fitness))
    appendToFile(str(prefix) + "_ShForces.txt", str(force))
    appendToFile(str(prefix) + "_ShMoments.txt", str(moment))

#### Objective Function

In [3]:
def objective_function(s, ShF, ShM, beta): #c is a constant that distributes the weight among the functions.
    return beta*ShF(s) + (1-beta)*ShM(s)

In [4]:
# Bounds for each variable
nVar = 5
bounds = []
for i in range(1,nVar*3+1):
    if(i%3==0): bounds.append([0.005,0.04])
    else: bounds.append([-0.16, 0.16])
bounds = np.array(bounds)
print('bounds',bounds)

bounds [[-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]
 [-0.16   0.16 ]
 [-0.16   0.16 ]
 [ 0.005  0.04 ]]


#### GD

In [5]:
#################### Descenso de Gradiente con diferenciación finita
def Gradient_Descent(X0,f,bounds,MaxIter=1000,alpha=1e-3, args = ()):
    eps = 1e-5
    def Gradiente(X,f, args = ()):
        n = len(X)
        G = np.zeros((n),float)
        incX = np.zeros((n),float)
        for i in range(n):
            incX[i] = eps
            G[i] = (f(X+incX, *args)-f(X, *args))/eps
            incX[i] = 0
        return G
    def getStepSize(alpha,m,X,P,G,f, args = ()):
        c0 = 1e-4
        c1 = 2
        c2 = 5
        c3 = 3
        eps = 1e-8
        while f(X+alpha*P, *args) > f(X, *args)+c0*np.dot(G,P):
            m = 0
            alpha = alpha/c1
            if alpha<=eps:
                break
        m += 1
        if m>=c2:
            m=0
            alpha = c3*alpha
        return alpha,m

    k=0
    X = X0
    G = Gradiente(X,f, args)
    normaGradiente = np.linalg.norm(G)
    m = 0
    while(k<=MaxIter and normaGradiente>=eps):
        G = Gradiente(X0,f, args)
        normaGradiente = np.linalg.norm(G)
        P = - G / normaGradiente
        alpha,m = getStepSize(alpha,m,X,P,G,f, args)
        X = X + alpha*P
        X = np.clip(X,bounds[:,0],bounds[:,1])
        k = k+1
        if k%100 == 0:
            print("\t\t#",k,f(X, *args))
    return X


In [6]:
def random_start(bounds):
    arr = np.zeros(bounds.shape[0])
    for i, tupl in enumerate(bounds):
        rand = np.random.uniform(tupl[0], tupl[1])
        arr[i] = rand 
    return arr
    
print(random_start(bounds))

[0.12173408 0.0228091  0.02986408 0.14795922 0.06884018 0.01371245
 0.06054481 0.08699946 0.0253125  0.04012633 0.11779958 0.0115934
 0.00263805 0.10570684 0.03578206]


In [7]:
# it = 80
# n = 4
def GD(GD_alph, iter, nRuns):
    startTime_GD = str(int(time.time()))
    eTime = 0

    print(f'\n* Number of iterations: {iter}')
    for i in range(nRuns):
        r = random_start(bounds)
        beta = np.random.normal(0.5, 0.20)
        fitness = objective_function(r, ShF, ShM, beta)

        print(f"\t\nInitial Fitness: {fitness} in #{i}")
        start = time.perf_counter()
        r = Gradient_Descent(r,objective_function, bounds,MaxIter=iter, alpha=GD_alph, args=(ShF, ShM, beta))
        end = time.perf_counter()
        fit_GD = objective_function(r,ShF,ShM, beta)
        print("\t  - before: ", fitness)
        print("\t  - after (GD): ", fit_GD)
        eTime += (end-start) #Time in seconds
        shF, shM = ShF(r), ShM(r)
        if shF < 1 and shM < 1:
            logSample(startTime_GD, r, fit_GD, shF, shM) # (now, sample, fitness, force, moment):
    if nRuns: eTime /= nRuns
    print("Average time of execution:", eTime,"seconds. It was run", nRuns, "times.")
    # return fit_GD

In [13]:
it = 1000
nRuns = 100

In [9]:
GD(1e-5, it, nRuns)


* Number of iterations: 100
	
Initial Fitness: 47.625849787495305 in #0
		# 100 47.62582401531999
	  - before:  47.625849787495305
	  - after (GD):  47.62582401531999
	
Initial Fitness: 56.92045515452459 in #1
		# 100 56.92040421778029
	  - before:  56.92045515452459
	  - after (GD):  56.92040421778029
	
Initial Fitness: 26.752624589528004 in #2
		# 100 26.75260770953297
	  - before:  26.752624589528004
	  - after (GD):  26.75260770953297
Average time of execution: 57.52570926666764 seconds. It was run 3 times.
None


In [10]:
GD(1e-3, it, nRuns)


* Number of iterations: 100
	
Initial Fitness: 32.15063658129415 in #0
		# 100 3.0478674607341825
	  - before:  32.15063658129415
	  - after (GD):  3.0478674607341825
	
Initial Fitness: 50.05931740819348 in #1
		# 100 15.470005377868148
	  - before:  50.05931740819348
	  - after (GD):  15.469837865014858
	
Initial Fitness: 25.843819490431525 in #2
		# 100 12.999227137518837
	  - before:  25.843819490431525
	  - after (GD):  13.02234353901232
Average time of execution: 59.52448293333388 seconds. It was run 3 times.
None


In [11]:
GD(1e-8, it, nRuns)


* Number of iterations: 100
	
Initial Fitness: 76.12453805064486 in #0
		# 100 76.12451677097917
	  - before:  76.12453805064486
	  - after (GD):  76.12451677097917
	
Initial Fitness: 111.47306877825972 in #1
		# 100 111.47304344565731
	  - before:  111.47306877825972
	  - after (GD):  111.47304344565731
	
Initial Fitness: 50.69712288978097 in #2
		# 100 50.69710729804423
	  - before:  50.69712288978097
	  - after (GD):  50.69710729804423
Average time of execution: 60.0233203000001 seconds. It was run 3 times.
None


In [14]:
GD(1e-4, it, nRuns)


* Number of iterations: 1000
	
Initial Fitness: 5.3856679654383655 in #0
		# 100 5.385665993263544
		# 200 5.385665993263544
		# 300 5.385665993263544
		# 400 5.385665993263544
		# 500 5.385665993263544


KeyboardInterrupt: 

In [15]:
GD(3e-3, it, nRuns)


* Number of iterations: 1000
	
Initial Fitness: 53.616266621359 in #0
		# 100 15.945683536291718
		# 200 15.489373908412515
		# 300 15.441371310591427
		# 400 15.381178143820135
		# 500 15.469948761362016
		# 600 16.170942943085894
		# 700 17.030309304231317
		# 800 18.839632444222037
		# 900 21.86035656254319
		# 1000 22.258727130499622
	  - before:  53.616266621359
	  - after (GD):  22.272478251603633
	
Initial Fitness: 29.523204171785466 in #1
		# 100 10.474672590477507
		# 200 10.474672590477507
		# 300 10.474672590477507
		# 400 10.474672590477507
		# 500 10.474672590477507
		# 600 10.474672590477507
		# 700 10.474672590477507
		# 800 10.474672590477507
		# 900 10.474672590477507
		# 1000 10.474672590477507
	  - before:  29.523204171785466
	  - after (GD):  10.474672590477507
	
Initial Fitness: 25.446676801562347 in #2
		# 100 27.339807038403812
		# 200 31.8505861906462
		# 300 40.460797169775645
		# 400 40.80547393326732
		# 500 43.193140875825534
		# 600 43.80123555696948
		# 7